# 1 - Embedding

Agora, em vez de substituir cada palavra por um número, iremos identificar as palavras como vetores em um espaço de n-dimensões.

O processo de Embedding é fazer como um conjunto de palavras ou palavras associadas formem vetores em um espaço multidimensional

Nesse semana de curso, será usado o dataset do IMDB, para podermos construir um classificador de críticas em relação a um filme.

In [2]:
!pip install -q tensorflow-datasets

In [11]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd
#print(tf.__version__)

In [4]:
imdb, info = tfds.load('imdb_reviews', with_info=True, as_supervised=True)

Dl Completed...: 100%|██████████| 1/1 [05:21<00:00, 321.18s/ url]


Dataset imdb_reviews downloaded and prepared to C:\Users\kaisson.ferreira\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


In [8]:
train_data, test_data = imdb['train'], imdb['test']

In [14]:
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

for l,s in train_data:
    training_sentences.append(str(s.numpy()))
    training_labels.append(l.numpy())


for l,s in test_data:
    testing_sentences.append(str(s.numpy()))
    testing_labels.append(l.numpy())